
# **Mount and Installs**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow
!pip install vit-keras
!pip install tensorflow-addons

# **Attention Network**

In [ ]:
#attention  networks model:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.utils import resample
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Reshape, Attention, Bidirectional, LSTM, GlobalAveragePooling1D


shot_types = ['sweep', 'drive', 'pull', 'slog', 'flick', 'cut']

sample_fraction = 0.05

data = []
labels = []

for label, shot_type in enumerate(shot_types):
    folder_path = f'/content/drive/MyDrive/Datasets/Dataset_1_40/{shot_type}'

    # equal representation of each class
    sss = StratifiedShuffleSplit(n_splits=1, test_size=sample_fraction, random_state=42)

    for _, sampled_indices in sss.split(np.zeros(len(os.listdir(folder_path))), label * np.ones(len(os.listdir(folder_path)))):
        sampled_files = [os.listdir(folder_path)[i] for i in sampled_indices]

        for vid_file in sampled_files:
            vid_path = os.path.join(folder_path, vid_file)
            vid = cv2.VideoCapture(vid_path)

            while True:
                ret, frame = vid.read()
                if not ret:
                    break
                frame = cv2.resize(frame, (100, 100))
                data.append(frame)
                labels.append(label)

            vid.release()

data = np.array(data) / 255.0
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


def build_attention_model(input_shape, num_classes):
    input_layer = Input(shape=input_shape)

    # Convolutional layers
    x = Conv2D(32, (3, 3), activation='relu')(input_layer)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)

    # Reshape for time-distributed layer
    x = Reshape((1, -1))(x)

    # LSTM layer with attention mechanism
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    attention = Attention()([x, x])

    # Global average pooling
    x = GlobalAveragePooling1D()(attention)

    # Dense layers
    x = Dense(128, activation='relu')(x)
    output_layer = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

#input shape and number of classes
input_shape = X_train[0].shape
num_classes = len(np.unique(y_train))

attention_model = build_attention_model(input_shape, num_classes)

attention_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


attention_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=4, batch_size=32)

# model_save_path = '/content/drive/MyDrive/Datasets/Datasets/Shots/attention_model.h5'
# attention_model.save(model_save_path)

# Evaluate the model on the test set
accuracy = attention_model.evaluate(X_test, y_test)[1]
print('Test Accuracy:', accuracy)

Epoch 1/4
52/52 [==============================] - 11s 71ms/step - loss: 1.4335 - accuracy: 0.4400 - val_loss: 0.7817 - val_accuracy: 0.7567
Epoch 2/4
52/52 [==============================] - 2s 35ms/step - loss: 0.4124 - accuracy: 0.8631 - val_loss: 0.1624 - val_accuracy: 0.9465
Epoch 3/4
52/52 [==============================] - 2s 34ms/step - loss: 0.0387 - accuracy: 0.9903 - val_loss: 0.0349 - val_accuracy: 0.9951
Epoch 4/4
13/13 [==============================] - 0s 11ms/step - loss: 0.0270 - accuracy: 0.9951
Test Accuracy: 0.9951338171958923


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming attention_model.predict returns the predicted probabilities for each class
y_pred_probabilities = attention_model.predict(X_test)
# Convert probabilities to class predictions
y_pred = np.argmax(y_pred_probabilities, axis=1)

# Calculate precision, recall, and f1 score
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
accuracy = attention_model.evaluate(X_test, y_test)[1]
print('Test Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

13/13 [==============================] - 0s 11ms/step - loss: 0.0270 - accuracy: 0.9951
Test Accuracy: 0.9951338171958923
Precision: 0.995183474849794
Recall: 0.9951338199513382
F1 Score: 0.9951276455753844


# **CNN+RNN**

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, LSTM, Bidirectional, GlobalAveragePooling1D, Reshape

shot_types = ['sweep', 'drive', 'pull', 'slog', 'flick', 'cut']

sample_fraction = 0.8

data = []
labels = []

for label, shot_type in enumerate(shot_types):
    # folder_path = f'/content/drive/MyDrive/Datasets/Datasets/Shots/{shot_type}'
    folder_path = f'/content/drive/MyDrive/Dataset_1_40/{shot_type}'

    # equal representation of each class
    sss = StratifiedShuffleSplit(n_splits=1, test_size=sample_fraction, random_state=42)

    for _, sampled_indices in sss.split(np.zeros(len(os.listdir(folder_path))), label * np.ones(len(os.listdir(folder_path)))):
        sampled_files = [os.listdir(folder_path)[i] for i in sampled_indices]

        for vid_file in sampled_files:
            vid_path = os.path.join(folder_path, vid_file)
            vid = cv2.VideoCapture(vid_path)

            while True:
                ret, frame = vid.read()
                if not ret:
                    break
                frame = cv2.resize(frame, (100, 100))
                data.append(frame)
                labels.append(label)

            vid.release()

data = np.array(data) / 255.0
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

def build_cnn_rnn_model(input_shape, num_classes):
    input_layer = Input(shape=input_shape)

    # Convolutional layers
    x = Conv2D(32, (3, 3), activation='relu')(input_layer)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)

    # Reshape for time-distributed layer
    x = Reshape((1, -1))(x)

    # Bidirectional LSTM layer
    x = Bidirectional(LSTM(128, return_sequences=True))(x)

    # Global average pooling
    x = GlobalAveragePooling1D()(x)

    # Dense layers
    x = Dense(128, activation='relu')(x)
    output_layer = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

# Input shape and number of classes
input_shape = X_train[0].shape
num_classes = len(np.unique(y_train))

cnn_rnn_model = build_cnn_rnn_model(input_shape, num_classes)

cnn_rnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

cnn_rnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)

# model_save_path = '/content/drive/MyDrive/Shots/cnn_rnn_model.h5'
# cnn_rnn_model.save(model_save_path)

# Evaluate the model on the test set
accuracy = cnn_rnn_model.evaluate(X_test, y_test)[1]
print('Test Accuracy:', accuracy)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming attention_model.predict returns the predicted probabilities for each class
y_pred_probabilities = cnn_rnn_model.predict(X_test)

# Convert probabilities to class predictions
y_pred = np.argmax(y_pred_probabilities, axis=1)

# Calculate precision, recall, and f1 score
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

# **ViT Model 1**

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from vit_keras import vit

# Define shot types and other parameters
#shot_types = ['drive', 'pull']
shot_types=['sweep', 'drive', 'pull', 'slog', 'flick', 'cut']
sample_fraction = 0.4

data = []
labels = []

for label, shot_type in enumerate(shot_types):
    folder_path = f'/content/drive/MyDrive/Dataset_1_40/{shot_type}'

    # Equal representation of each class
    sss = StratifiedShuffleSplit(n_splits=1, test_size=sample_fraction, random_state=42)

    for _, sampled_indices in sss.split(np.zeros(len(os.listdir(folder_path))), label * np.ones(len(os.listdir(folder_path)))):
        sampled_files = [os.listdir(folder_path)[i] for i in sampled_indices]

        for vid_file in sampled_files:
            vid_path = os.path.join(folder_path, vid_file)
            vid = cv2.VideoCapture(vid_path)

            while True:
                ret, frame = vid.read()
                if not ret:
                    break
                frame = cv2.resize(frame, (96, 96))  # Adjust the size based on the ViT model requirements
                data.append(frame)
                labels.append(label)

            vid.release()

data = np.array(data) / 255.0
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Load ViT model without classification head
vit_model = vit.vit_l32(
    image_size=(96, 96),
    activation='softmax',
    pretrained=True,
    include_top=False,
    pretrained_top=False,
    classes=len(shot_types)
)

# Build the new model by adding a classification head
input_layer = Input(shape=(96, 96, 3))
x = vit_model(input_layer)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
output_layer = Dense(len(shot_types), activation='softmax')(x)

vit_model_with_head = Model(inputs=input_layer, outputs=output_layer)

# Compile and train the model
vit_model_with_head.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
vit_model_with_head.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=6)

# Save the model
# model_save_path = '/content/drive/MyDrive/Shots/vit_model.h5'
# vit_model_with_head.save(model_save_path)

# Evaluate the model on the test set
accuracy = vit_model_with_head.evaluate(X_test, y_test)[1]
print('Test Accuracy:', accuracy)

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


1226658854/1226658854 [==============================] - 14s 0us/step


/usr/local/lib/python3.10/dist-packages/vit_keras/utils.py:81: UserWarning: Resizing position embeddings from 12, 12 to 3, 3
  warnings.warn(


Epoch 1/5
616/616 [==============================] - 5733s 9s/step - loss: 1.6757 - accuracy: 0.2849 - val_loss: 1.6867 - val_accuracy: 0.2947
Epoch 2/5
333/616 [===============>..............] - ETA: 40:57 - loss: 1.6596 - accuracy: 0.2768

KeyboardInterrupt: ignored

# **ViT Model 2**

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, LSTM, Bidirectional, GlobalAveragePooling1D, Reshape
from tensorflow.keras.optimizers import Adam
from vit_keras import vit
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define shot types and other parameters
shot_types = ['sweep', 'drive', 'pull', 'slog', 'flick', 'cut']
sample_fraction = 0.4

data = []
labels = []

for label, shot_type in enumerate(shot_types):
    folder_path = f'/content/drive/MyDrive/Shots/{shot_type}'

    # Equal representation of each class
    sss = StratifiedShuffleSplit(n_splits=1, test_size=sample_fraction, random_state=42)

    for _, sampled_indices in sss.split(np.zeros(len(os.listdir(folder_path))), label * np.ones(len(os.listdir(folder_path)))):
        sampled_files = [os.listdir(folder_path)[i] for i in sampled_indices]

        for vid_file in sampled_files:
            vid_path = os.path.join(folder_path, vid_file)
            vid = cv2.VideoCapture(vid_path)

            while True:
                ret, frame = vid.read()
                if not ret:
                    break
                frame = cv2.resize(frame, (96, 96))
                data.append(frame)
                labels.append(label)

            vid.release()

data = np.array(data) / 255.0
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Create ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    # rotation_range=20,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # shear_range=0.2,
    # zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load ViT model without classification head
vit_model = vit.vit_l32(
    image_size=(96, 96),
    activation='softmax',
    pretrained=True,
    include_top=False,
    pretrained_top=False,
    classes=len(shot_types)
)

# Build the new model by adding a classification head
input_layer = Input(shape=(96, 96, 3))
x = vit_model(input_layer)
x = Flatten()(x)

# Reshape for time-distributed layer
x = Reshape((1, -1))(x)

# Bidirectional LSTM layer
x = Bidirectional(LSTM(128, return_sequences=True))(x)

# Global average pooling
x = GlobalAveragePooling1D()(x)

x = Dense(256, activation='relu')(x)
output_layer = Dense(len(shot_types), activation='softmax')(x)

vit_model_with_head = Model(inputs=input_layer, outputs=output_layer)

# Compile and train the model using the generator
vit_model_with_head.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
vit_model_with_head.fit(datagen.flow(X_train, y_train, batch_size=32),
                                  steps_per_epoch=len(X_train) // 32,
                                  validation_data=(X_test, y_test),
                                  epochs=4)

# Save the model
model_save_path = '/content/drive/MyDrive/Shots/vit_model_with_generator.h5'
vit_model_with_head.save(model_save_path)

# Evaluate the model on the test set
accuracy = vit_model_with_head.evaluate(X_test, y_test)[1]
print('Test Accuracy:', accuracy)

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


1226658854/1226658854 [==============================] - 7s 0us/step


/usr/local/lib/python3.10/dist-packages/vit_keras/utils.py:81: UserWarning: Resizing position embeddings from 12, 12 to 3, 3
  warnings.warn(


Epoch 1/4
115/115 [==============================] - 148s 417ms/step - loss: 1.6650 - accuracy: 0.2787 - val_loss: 1.6821 - val_accuracy: 0.2557
Epoch 2/4
115/115 [==============================] - 39s 337ms/step - loss: 1.6551 - accuracy: 0.2790 - val_loss: 1.6780 - val_accuracy: 0.2557
Epoch 3/4
115/115 [==============================] - 45s 391ms/step - loss: 1.6511 - accuracy: 0.2863 - val_loss: 1.6795 - val_accuracy: 0.2947
Epoch 4/4
115/115 [==============================] - 42s 367ms/step - loss: 1.6524 - accuracy: 0.2721 - val_loss: 1.6766 - val_accuracy: 0.2947


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


29/29 [==============================] - 5s 162ms/step - loss: 1.6766 - accuracy: 0.2947
Test Accuracy: 0.2946912348270416


# **ViT Model 3 - With Generator and Edits (Faster)**

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, LSTM, Bidirectional, GlobalAveragePooling1D, Reshape
from tensorflow.keras.optimizers import Adam
from vit_keras import vit
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define shot types and other parameters
shot_types = ['sweep', 'drive', 'pull', 'slog', 'flick', 'cut']
sample_fraction = 0.05

data = []
labels = []

for label, shot_type in enumerate(shot_types):
    folder_path = f'/content/drive/MyDrive/Datasets/Dataset_1_40/{shot_type}'

    # Equal representation of each class
    sss = StratifiedShuffleSplit(n_splits=1, test_size=sample_fraction, random_state=42)

    for _, sampled_indices in sss.split(np.zeros(len(os.listdir(folder_path))), label * np.ones(len(os.listdir(folder_path)))):
        sampled_files = [os.listdir(folder_path)[i] for i in sampled_indices]

        for vid_file in sampled_files:
            vid_path = os.path.join(folder_path, vid_file)
            vid = cv2.VideoCapture(vid_path)

            while True:
                ret, frame = vid.read()
                if not ret:
                    break
                frame = cv2.resize(frame, (96, 96))
                data.append(frame)
                labels.append(label)

            vid.release()

data = np.array(data) / 255.0
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

#generator
datagen = ImageDataGenerator(
    # rotation_range=20,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # shear_range=0.2,
    # zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

#ViT Model
vit_model = vit.vit_l32(
    image_size=(96, 96),
    activation='softmax',
    pretrained=True,
    include_top=False,
    pretrained_top=False,
    classes=len(shot_types)
)

#classification head
input_layer = Input(shape=(96, 96, 3))
x = vit_model(input_layer)
x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Reshape((1, -1))(x)
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = GlobalAveragePooling1D()(x)
x = Dense(256, activation='relu')(x)
output_layer = Dense(len(shot_types), activation='softmax')(x)

vit_model_with_head = Model(inputs=input_layer, outputs=output_layer)

#compile train
vit_model_with_head.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
vit_model_with_head.fit(datagen.flow(X_train, y_train, batch_size=32),
                                  steps_per_epoch=len(X_train) // 32,
                                  validation_data=(X_test, y_test),
                                  epochs=4)

# Save
# model_save_path = '/content/drive/MyDrive/Shots/vit_model_with_generator.h5'
# vit_model_with_head.save(model_save_path)

# Evaluate
accuracy = vit_model_with_head.evaluate(X_test, y_test)[1]
print('Test Accuracy:', accuracy)

Epoch 1/4
51/51 [==============================] - 8s 53ms/step - loss: 1.6705 - accuracy: 0.2713 - val_loss: 1.4266 - val_accuracy: 0.3990
Epoch 2/4
51/51 [==============================] - 2s 30ms/step - loss: 1.0441 - accuracy: 0.5953 - val_loss: 0.6859 - val_accuracy: 0.7397
Epoch 3/4
51/51 [==============================] - 2s 31ms/step - loss: 0.3429 - accuracy: 0.8870 - val_loss: 0.1523 - val_accuracy: 0.9586
Epoch 4/4
13/13 [==============================] - 0s 12ms/step - loss: 0.0779 - accuracy: 0.9854
Test Accuracy: 0.985401451587677


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming attention_model.predict returns the predicted probabilities for each class
y_pred_probabilities = vit_model_with_head.predict(X_test)

# Convert probabilities to class predictions
y_pred = np.argmax(y_pred_probabilities, axis=1)

# Calculate precision, recall, and f1 score
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

13/13 [==============================] - 1s 9ms/step
Precision: 0.9856642813238483
Recall: 0.9854014598540146
F1 Score: 0.9853736429410264


# **ResNet50**

In [ ]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Input
from tensorflow.keras.applications import ResNet50
import time
data = []
labels = []
desired_frame_count = 5
#, 'cut', 'slog', 'sweep', 'flick', 'misc'
shot_types = ['drive','pull']

for label, shot_type in enumerate(shot_types):
    folder_path = f'/content/drive/MyDrive/Capstone 2023-24 SRU 02/Datasets/Shots/{shot_type}'
    for vid_file in os.listdir(folder_path):
        vid_path = os.path.join(folder_path, vid_file)
        vid = cv2.VideoCapture(vid_path)
        print('Processing video:', vid_file)
        total_frames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
        middle_frame = total_frames // 2
        frame_count = 0
        while True:
          ret, frame = vid.read()
          if not ret:
              break
          frame_count +=1
          if frame_count < middle_frame - desired_frame_count // 2:
              continue
          if frame_count > middle_frame + desired_frame_count // 2:
              break

          frame = cv2.resize(frame, (100, 100))
          rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB
          data.append(rgb_frame)
          labels.append(label)
        vid.release()
data = np.array(data) / 255.0
labels = np.array(labels)
X_train, X_test, y_train, y_test = train_test_split(data, labels, train_size=0.75, test_size=0.25, random_state=42)
# ResNet model
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(100, 100, 3))

# Freeze the layers of the ResNet model
for layer in resnet.layers[:-2]:
    layer.trainable = True
for layer in resnet.layers[-2:]:
    layer.trainable = False
# Create a new model and add the ResNet base
model = Sequential()
model.add(resnet)

# Add additional layers on top of the ResNet base
model.add(Flatten())
model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=len(shot_types), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=10, batch_size=32)
# Save
model_save_path = '/content/drive/MyDrive/Datasets/Shots/resnet+cnn.h5'
model.save(model_save_path)

# Evaluate
accuracy = model.evaluate(X_test, y_test)[1]
print('Test Accuracy:', accuracy)

25/25 [==============================] - 49s 219ms/step - loss: 2.7862 - accuracy: 0.5121 - val_loss: 0.6929 - val_accuracy: 0.6540
Epoch 2/10
25/25 [==============================] - 3s 115ms/step - loss: 0.5985 - accuracy: 0.6658 - val_loss: 0.6913 - val_accuracy: 0.6540
Epoch 3/10
25/25 [==============================] - 3s 117ms/step - loss: 0.4629 - accuracy: 0.7878 - val_loss: 0.7084 - val_accuracy: 0.3460
Epoch 4/10
25/25 [==============================] - 3s 112ms/step - loss: 0.3649 - accuracy: 0.8628 - val_loss: 2.8812 - val_accuracy: 0.3460
Epoch 5/10
25/25 [==============================] - 3s 113ms/step - loss: 0.3012 - accuracy: 0.8882 - val_loss: 0.8255 - val_accuracy: 0.3460
Epoch 6/10
25/25 [==============================] - 3s 113ms/step - loss: 0.2228 - accuracy: 0.9174 - val_loss: 1.2035 - val_accuracy: 0.3460
Epoch 7/10
25/25 [==============================] - 3s 114ms/step - loss: 0.3041 - accuracy: 0.8767 - val_loss: 0.6479 - val_accuracy: 0.6540
Epoch 8/10
25/25

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


9/9 [==============================] - 0s 36ms/step - loss: 0.6501 - accuracy: 0.6540
Test Accuracy: 0.6539924144744873


In [ ]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Input
from tensorflow.keras.applications import ResNet50
import time
data = []
labels = []
desired_frame_count = 5
#, 'cut', 'slog', 'sweep', 'flick', 'misc'
shot_types = ['drive', 'pull', 'cut', 'slog', 'sweep', 'flick']

for label, shot_type in enumerate(shot_types):
    folder_path = f'/content/drive/MyDrive/Capstone 2023-24 SRU 02/Datasets/Shots/{shot_type}'
    for vid_file in os.listdir(folder_path):
        vid_path = os.path.join(folder_path, vid_file)
        vid = cv2.VideoCapture(vid_path)
        print('Processing video:', vid_file)
        total_frames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
        middle_frame = total_frames // 2
        frame_count = 0
        while True:
          ret, frame = vid.read()
          if not ret:
              break
          frame_count +=1
          if frame_count < middle_frame - desired_frame_count // 2:
              continue
          if frame_count > middle_frame + desired_frame_count // 2:
              break

          frame = cv2.resize(frame, (100, 100))
          rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB
          data.append(rgb_frame)
          labels.append(label)
        vid.release()
data = np.array(data) / 255.0
labels = np.array(labels)
X_train, X_test, y_train, y_test = train_test_split(data, labels, train_size=0.75, test_size=0.25, random_state=42)
# ResNet model
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(100, 100, 3))

# Freeze the layers of the ResNet model
for layer in resnet.layers[:-2]:
    layer.trainable = True
for layer in resnet.layers[-2:]:
    layer.trainable = False
# Create a new model and add the ResNet base
model = Sequential()
model.add(resnet)

# Add additional layers on top of the ResNet base
model.add(Flatten())
model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=len(shot_types), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=10, batch_size=32)
# Save
model_save_path = '/content/drive/MyDrive/Datasets/Shots/resnet+cnn.h5'
model.save(model_save_path)

# Evaluate
accuracy = model.evaluate(X_test, y_test)[1]
print('Test Accuracy:', accuracy)

Epoch 1/10
53/53 [==============================] - 39s 168ms/step - loss: 3.4820 - accuracy: 0.3306 - val_loss: 1.6991 - val_accuracy: 0.3156
Epoch 2/10
53/53 [==============================] - 6s 116ms/step - loss: 1.4598 - accuracy: 0.3962 - val_loss: 1.7634 - val_accuracy: 0.2429
Epoch 3/10
53/53 [==============================] - 6s 112ms/step - loss: 1.2909 - accuracy: 0.4938 - val_loss: 1.7069 - val_accuracy: 0.1631
Epoch 4/10
53/53 [==============================] - 6s 117ms/step - loss: 1.0409 - accuracy: 0.5872 - val_loss: 1.7836 - val_accuracy: 0.1613
Epoch 5/10
53/53 [==============================] - 6s 118ms/step - loss: 0.8412 - accuracy: 0.6931 - val_loss: 1.8626 - val_accuracy: 0.1170
Epoch 6/10
53/53 [==============================] - 6s 114ms/step - loss: 0.6860 - accuracy: 0.7569 - val_loss: 1.8174 - val_accuracy: 0.1454
Epoch 7/10
53/53 [==============================] - 6s 119ms/step - loss: 0.5134 - accuracy: 0.8143 - val_loss: 4.1815 - val_accuracy: 0.1613
Epoch